In [1]:
from general import *

In [2]:
class PostureNeuron():
    NeuronState = namedtuple('NeuronState', ['V', 'w', 'spk', 'I'])

    def __init__(self, parameters):
        super(PostureNeuron, self).__init__()
        self.tau = parameters['tau']
        self.V_R = parameters['V_R']
        self.V_T = parameters['V_T']
        self.w = parameters['w']
        self.n = parameters['n']
        self.N_input = parameters['N_input']
        self.dt = parameters['dt']
        self.state = None

    def initialize_state(self):
        self.state = None

    def forward(self, input):
        if self.state is None:
            self.state = self.NeuronState(V=np.full((self.n,), self.V_R),
                                          w=np.array(self.w),
                                          spk=np.zeros(self.n),
                                          I=np.zeros((self.n, self.N_input)))
            
        V = self.state.V
        w = self.state.w
        I = self.state.I
        
        V += self.dt * (self.V_R - V) / self.tau
        V += np.sum(w * input, axis=-1)

        spk = np.heaviside(V - self.V_T, 0)
        V = (1 - spk) * V + spk * self.V_R

        self.state = self.NeuronState(V=V, w=w, spk=spk, I=I)

        return V, spk

In [3]:
with open('data/spike_primitive', 'rb') as file:
    spike_primitive = pickle.load(file) 
    
with open('data/joint_angles_step', 'rb') as file:
    joint_angles_step = pickle.load(file)
    
primitive_parameters = {'tau': 0.5e-3, 'V_T': -50e-3, 'V_R': -70e-3, 'n': len(synapse_type), 'w': weights_primitive, 'N_input': 3, 'dt': constants['dt']}

FileNotFoundError: [Errno 2] No such file or directory: 'data/joint_angles_step'